In [2]:
import torch
import transformers
from transformers import pipeline
from transformers import DistilBertModel, DistilBertForMaskedLM, DistilBertTokenizer
from optimum.quanto import freeze, quantize, qint8
import datasets
from transformers import TrainingArguments
import numpy as np
import evaluate

2025-02-24 16:08:02.643611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model_id = "distilbert-base-uncased"
#model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased", torch_dtype=torch.float16, attn_implementation="sdpa")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
dataset = datasets.load_dataset("imdb")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [5]:
def preprocess(data):
    tokens = tokenizer(data["text"], truncation=True, padding = 'max_length',  max_length=512)
    tokens["label"] = data["label"]
    return tokens

In [6]:
tokens = dataset.map(preprocess, batched = True)

In [7]:
labels = tokens['train'].features['label'].names
num_labels = len(labels)
label2id, id2label = {}, {}

for idx, lbl in enumerate(labels):
    label2id[lbl] = idx
    id2label[idx] = lbl

In [8]:
small_train_dataset = tokens["train"].shuffle(seed=11).select(range(2000))
small_eval_dataset = tokens["train"].shuffle(seed=11).select(range(2000))

In [9]:
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification, DistilBertConfig, DataCollatorWithPadding

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels = num_labels,
    id2label = id2label,
    label2id = label2id,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def preprocess_logits_for_metrics(logits, labels):
    """
    Preprocess the logits to ensure they are in the correct format for metric computation.
    This function will be called during the evaluation process.
    """
    if isinstance(logits, tuple):  
        logits = logits[0]  # get logit tensors

    pred_ids = torch.argmax(logits, dim=-1)
    
    return pred_ids, labels
    
def compute_metrics(eval_pred):
    
    predictions, labels = eval_pred

    return accuracy.compute(predictions=predictions[0], references=labels)


In [16]:
from transformers import TrainingArguments, Trainer
from optimum.intel import INCModelForSequenceClassification, INCTrainer
from neural_compressor import QuantizationAwareTrainingConfig

EPOCHS = 1
BATCH_SIZE = 16
LEARNING_RATE = 0.00005

training_args = TrainingArguments(
    output_dir = './imdb_tune_distilbert',
    num_train_epochs = EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LEARNING_RATE,
    logging_dir = './logs',
    load_best_model_at_end= True,
    metric_for_best_model="accuracy",
    eval_strategy="epoch",
    eval_steps = 500,
    save_strategy="epoch",
    save_total_limit=2,
    report_to=['tensorboard'],
)

quantization_config = QuantizationAwareTrainingConfig()
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = INCTrainer(
    model=model,  
    quantization_config=quantization_config,                       
    args=training_args,                  
    train_dataset=small_train_dataset,         
    eval_dataset=small_eval_dataset.shuffle(seed=72).select(range(600)),
    compute_metrics = compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
    tokenizer = tokenizer,
    data_collator = data_collator,
)


RuntimeError: Failed to import optimum.intel.neural_compressor because of the following error (look up to see its traceback):
cannot import name 'IncConfig' from 'optimum.intel.neural_compressor.configuration' (/home/se/coding/school/exjobb/mycode/.venv/lib/python3.12/site-packages/optimum/intel/neural_compressor/configuration.py)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.182836,0.941500


TrainOutput(global_step=125, training_loss=0.39196124267578125, metrics={'train_runtime': 2017.9639, 'train_samples_per_second': 0.991, 'train_steps_per_second': 0.062, 'total_flos': 264934797312000.0, 'train_loss': 0.39196124267578125, 'epoch': 1.0})

In [ ]:
""" untrained = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels = num_labels,
    id2label = id2label,
    label2id = label2id,
)

training_args_untrained = TrainingArguments(
    output_dir="./results",  
    per_device_eval_batch_size=2, 
    use_cpu=True,  
    no_cuda = True,
    logging_dir="./logs",  
    logging_steps=10,
)

trainer_untrained = Trainer(
    model=untrained,
    args=training_args_untrained,
    eval_dataset=small_eval_dataset.select(range(600)),
    data_collator = data_collator,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
    compute_metrics = compute_metrics,
)

print("accuracy no training: ", trainer_untrained.evaluate())
 """

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/se/coding/school/exjobb/mycode/.venv/lib/python3.12/site-packages/transformers/training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Accuracy after quantization: 0.48833333333333334


quantize(model)
trainer.train()
freeze(model)
trainer.evaluate()